# Introdução

Esse caderno tem por objetivo a criação de um modelo básico de treinamento, utilizando o classificador [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), que faça a predição de homologação de arquivamentos de procedimentos enviados à 1A.CAM do MPF.

Nesse modelo vamos passar a considerar os textos das íntegras.

A primeira passagem, serão criadas colunas com as palavras-chaves informadas pela área negocial que vão indicar a ausência ou presença do termo no texto da íntegra da peça de promoção do arquivamento.

Na segunda passagem, vamos passar a considerar unicamente o texto da íntegra.

**Nota**: os dados desse modelo foram recuperados de procedimentos que tiveram suas deliberações realizadas após o dia 02/07/2018, data em que a nova composição tomou posse na 1A.CAM.

# Carga de dados e pré-processamento

Vamos fazer a carga dos dados.

Vamos ler os textos das íntegras, limpá-los e associar ao dataframe com os dados de homologação.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

from sklearn import metrics

In [3]:
PATH = "../data/"

In [4]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/andrethiago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/andrethiago/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
# carga dos textos

from os import listdir
from os.path import isfile, join
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = stopwords.words('portuguese')
stop_words.extend(['n', 'nº', 'n°', 'n.º', ',', '.', '!', '?', ';', ':', '...', 'º', '–', '/', '(', ')'])

def get_text(file):
    with open(file, encoding='utf-8') as f:
        data=' '.join(line.strip() for line in f)
        f.close()
    
    return data.strip()

def get_text_non_stop_words(text):
    filtered_text = ''
    for w in word_tokenize(text, language='portuguese'):
        if w not in stop_words:
            filtered_text += w
            filtered_text += ' '            
    return filtered_text.strip()   
    

folder_integras = f'{PATH}/integras-textos'

texts = {}
for file in listdir(folder_integras):
    if isfile(join(folder_integras, file)):
        texts[file.split('.')[0]] = get_text(join(folder_integras,file))
    else:
        print('is not file', file)

    
texts['71078312']

'OFÍCIO MPF/PRJF/PROC/GAB Nº    Juiz de Fora, 28 de setembro de 2001   Procedimento Preparatório Autos n. 1.22.024.000066/2016-84        PROMOÇÃO DE ARQUIVAMENTO    Trata-se procedimento preparatório instaurado tendo em vista o encaminhamento, pela Procuradoria da República no Município de Viçosa, de duas representações, as quais narram possíveis irregularidades na condução do concurso para professor de nível médio, na modalidade de Educação à Distância, do Instituto Federal Sudeste de Minas Gerais, no campus Rio Pomba/MG (fls. 05 e 07).  De acordo com a representação, a banca do concurso teria modificado o seu “resultado preliminar” durante o próprio prazo de impugnação, o que inviabilizou a insurgência de candidatos que teriam sido anteriormente aprovados, mas foram desclassificados em virtude dessas alterações. Outro problema seria a necessidade de apresentação, ou não, no momento da inscrição, do documento constante no Anexo II, do mesmo Edital, referente à “declaração de disponibi

In [13]:
df_original[df_original['id'] == 71078312]

,areaAtuacao,classe,dataAutuacao,dataEntrada,homologado,id,identificadorPecaPromocao,itemCnmp,membroResponsavel,municipio,prioritario,procedimento,quantidadeConversoes,quantidadeProvidencias,textosProvidencias,urgente
4593,2,2,20-04-2016,08-11-2016,1,71078312,72932890.0,1854,8331.0,2775,0,1.22.024.000066/2016-84,0,6,"Junte-se.,Oficie-se.,Encaminhe-se o AA para qu...",0


In [14]:
# montando um DataFrame
df_original = pd.read_json(f'{PATH}/1A.CAM.homologacao-arquivamento.json')
df_work = df_original.copy()
#df_work.dropna(subset=['identificadorPecaPromocao'], inplace=True)
#df_work.reset_index(drop=True, inplace=True)
cols = ['id', 'homologado']
df_work.drop(inplace=True, columns=[col for col in df_work.columns if col not in cols])
df_work['peca_promocao'] = None
df_work.sample(5)

,homologado,id,peca_promocao
5328,1,68124308,None
3289,1,75189373,None
5354,1,47775742,None
4113,1,67477092,None
2725,1,69216388,None


In [15]:
# Associando o texto...
for key, text in zip(texts.keys(), texts.values()):
    if key != '' and text != '':
        df_work.loc[df_work.id == int(key), 'peca_promocao'] = text
        
df_work.sample(5)

,homologado,id,peca_promocao
1879,1,81457889,None
138,1,63246528,EXCELENTÍSSIMO SENHOR JUIZ FEDERAL DA 5ª VARA ...
4034,1,64700136,None
119,1,80519448,PRM-JZN-CE-00005464/2018 MINISTÉRIO PÚBLICO ...
5840,1,66252594,Arquivamento do Procedimento Preparatório nº 1...


In [18]:
print(len(df_work[df_work['peca_promocao'].isnull()]))
df_work.dropna(subset=['peca_promocao'], inplace=True)
df_work.reset_index(drop=True, inplace=True)
print(len(df_work[df_work['peca_promocao'].isnull()]))

1401
0


In [19]:
print(len(df_work[df_work['homologado'] == 1]), len(df_work[df_work['homologado'] == 0]))

4742 73


In [20]:
#Carga das palavras chaves
palavras_chaves = pd.read_csv(f'{PATH}/1A.CAM.palavras-chaves.txt')
palavras_chaves

,palavras-chaves
0,AUSÊNCIA DE IRREGULARIDADES
1,IRREGULARIDADE SANADA
2,NÃO COMPROVAÇÃO
3,INEXISTÊNCIA
4,SOLUCIONADA
5,EXAURIMENTO
6,EXAURIDA
7,PERDA DE OBJETO
8,IRREGULARIDADE NÃO COMPROVADA
9,INEXISTÊNCIA DE INDÍCIOS DE IRREGULARIDADE


## Busca de palavras-chaves nos textos das peças de promoção

Dada a lista de palavras-chaves anteriormente listada, vamos procurá-la nos textos das íntegras.

Vamos criar uma coluna para cada uma das palavras-chaves e marcar com ausência ou presença da mesma no texto correspondente.

Vamos utilzar expressões regulares.

In [21]:
pcs = palavras_chaves['palavras-chaves']

# vou ignorar as colunas com o caractere '+' // PRECISAMOS TRATAR ISSO AINDA (e ainda temos o problema dos espaços)
for pc in pcs:
    if pc.rfind('+') == -1:
        df_work[pc] = 0

df_work.sample(5)

,homologado,id,peca_promocao,AUSÊNCIA DE IRREGULARIDADES,IRREGULARIDADE SANADA,NÃO COMPROVAÇÃO,INEXISTÊNCIA,SOLUCIONADA,EXAURIMENTO,EXAURIDA,...,INEXISTÊNCIA DE INDÍCIOS MÍNIMOS DE IRREGULARIDADES,RECOMENDAÇÃO EXPEDIDA E ACATADA,JUSTIFICATIVAS SATISFATÓRIAS,CORREÇÃO DAS FALHAS,AÇÃO AJUIZADA,QUESTÃO JUDICIALIZADA,DIREITO INDIVIDUAL,EXCESSO DE PESO,MAMOGRAFIA,SISCAN
396,1,76707249,MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REP...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
634,1,77326677,PRM-UDI-MG-00007349/2018 Pelo falecimento da...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4573,1,47886797,PROMOÇÃO DE ARQUIVAMENTO Procedimento Prepara...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4353,1,67497530,Procedimento Administrativo n Ministério Púb...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1730,1,79558017,EXCELENTÍSSIMO SENHOR DOUTOR JUIZ FEDERAL DA 1...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
import re

for v in palavras_chaves['palavras-chaves']:
    for i in range(len(df_work)):
        #v_with_spaces = re.sub(' ', '\s+', v)
        if re.search(r"\b" + v + r"?\b", df_work['peca_promocao'][i], re.IGNORECASE):
            df_work.loc[i, v] = 1
            
df_work.sample(10)

,homologado,id,peca_promocao,AUSÊNCIA DE IRREGULARIDADES,IRREGULARIDADE SANADA,NÃO COMPROVAÇÃO,INEXISTÊNCIA,SOLUCIONADA,EXAURIMENTO,EXAURIDA,...,INEXISTÊNCIA DE INDÍCIOS MÍNIMOS DE IRREGULARIDADES,RECOMENDAÇÃO EXPEDIDA E ACATADA,JUSTIFICATIVAS SATISFATÓRIAS,CORREÇÃO DAS FALHAS,AÇÃO AJUIZADA,QUESTÃO JUDICIALIZADA,DIREITO INDIVIDUAL,EXCESSO DE PESO,MAMOGRAFIA,SISCAN
885,1,85741087,MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REP...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3340,1,55825221,Tutela coletiva – inquérito civil público Auto...,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1403,1,79250867,PRM-BGO-RS-00002166/2018 MINISTÉRIO PÚBLIC...,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1103,1,76300890,Inquérito Civil nº 1.14.007.000034/2017-40 PR...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2697,1,72006443,PRM-RSD-RJ-00001388/2017 Ref.: Inquérito Civ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2207,1,66672665,MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REP...,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1045,1,84518294,MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REP...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
204,1,64028799,Inquérito Civil nº 1.25.007.000068/2015-99 ...,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3140,1,69345137,MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REP...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4596,1,13764188,MINISTÉRIO PÚBLICO FEDERAL Procuradoria da Rep...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# vamos treinar o modelo
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df_work, random_state=42)
train.shape, valid.shape

((3611, 28), (1204, 28))

In [35]:
cols_to_remove = ['id', 'peca_promocao', 'homologado']
features = [col for col in df_work.columns if col not in cols_to_remove]

model = RandomForestClassifier(random_state=42)
model.fit(train[features], train['homologado'])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [36]:
model.score(df_work[features], df_work['homologado'])

0.9846313603322949

In [39]:
train_preds = model.predict(train[features])
valid_preds = model.predict(valid[features])

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def print_score():
    print('Accuracy score:', accuracy_score(valid['homologado'], valid_preds))
    print('Precision score: ', format(precision_score(valid['homologado'], valid_preds)))
    print('Recall score: ', format(recall_score(valid['homologado'], valid_preds)))
    print('F1 score: ', format(f1_score(valid['homologado'], valid_preds)))

    
print_score()

Accuracy score: 0.9842192691029901
Precision score:  0.9850374064837906
Recall score:  0.9991568296795953
F1 score:  0.9920468815403936


In [40]:
from sklearn.metrics import confusion_matrix

confusion_matrix(valid['homologado'], valid_preds).ravel()

array([   0,   18,    1, 1185], dtype=int64)